In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import itertools
from statsmodels.tsa.seasonal import seasonal_decompose

df = pd.read_csv('top20_ETFs.csv')
df_top20 = df.copy()

# IDEA A

In [2]:
# --- Instalación de Librerías ---
print("Instalando librerías necesarias (Prophet, Streamlit, Pyngrok)...")
!pip install prophet cmdstanpy streamlit pyngrok plotly -q # Añadido Plotly explícitamente
print("Instalación completada.")

# --- Imports para la Parte de Análisis ---
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt # Comentado - No se usa para plots aquí
import warnings
import gc
import itertools
import traceback
import pickle
import os
from datetime import timedelta # Añadido para cálculos de fechas

# Configuración de advertencias
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

# Imports de Prophet (verificar después de instalar)
try:
    from prophet import Prophet
    from prophet.diagnostics import cross_validation, performance_metrics
    print("Importaciones de Prophet correctas.")
except ImportError:
    print("*"*30); print("ERROR: Faltan componentes de Prophet. Intenta reiniciar y reinstalar."); print("*"*30); exit()

Instalando librerías necesarias (Prophet, Streamlit, Pyngrok)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.0 MB/s eta 0:00:00
Instalación completada.
Importaciones de Prophet correctas.


In [3]:
# --- 1. CONFIGURACIÓN GENERAL ---
TOP_N_ETFS = 20 # Procesar todos los ETFs encontrados
# !!! ASEGÚRATE DE QUE ESTA VARIABLE (DataFrame) EXISTA EN TU ENTORNO !!!
# Ejemplo: df_top20 = pd.read_csv('ETF_cohortes.csv')
DATASET_VARIABLE_NAME = 'df_top20' # Variable con datos de entrenamiento
FUTURE_END_DATE = '2030-01-01'      # Extender predicción más allá
ACTUAL_DATA_FILE = 'ETF_cohortes.csv' # Archivo para cargar datos (y regresores futuros si se usan)
FRONTEND_DATA_FILE = 'prophet_frontend_data_v2.pkl' # NUEVO nombre archivo salida

# --- 1.1 CONFIGURACIÓN DE MEJORAS ---
USE_LOG_TRANSFORM = True             # Mantenemos log-transform
REGRESSOR_COLUMNS = []
PERFORM_HYPERPARAMETER_TUNING = False # Poner a True para tuning (lento)
PARAM_GRID = { 'changepoint_prior_scale': [0.01, 0.1, 0.5], 'seasonality_prior_scale': [1.0, 10.0, 20.0], 'holidays_prior_scale': [1.0, 10.0, 20.0] }
PROPHET_BASE_PARAMS = { 'seasonality_mode': 'multiplicative', 'daily_seasonality': False, 'weekly_seasonality': True, 'yearly_seasonality': True, 'interval_width': 0.80 }
CUSTOM_HOLIDAYS = None

# --- 1.2 CONFIGURACIÓN CV ---
EJECUTAR_CROSS_VALIDATION = False     # Poner a True SOLO si haces tuning o quieres métricas CV
CV_INITIAL = '730 days'; CV_PERIOD = '90 days'; CV_HORIZON = '180 days'; CV_METRIC_TO_OPTIMIZE = 'rmse'

print("Configuración cargada.")

Configuración cargada.


In [4]:
# --- INICIO SCRIPT ANÁLISIS PROPHET v2 ---

# --- 2. CARGA Y PREPARACIÓN ---
print("\n--- Iniciando Análisis Prophet ---")
if DATASET_VARIABLE_NAME not in globals(): raise NameError(f"'{DATASET_VARIABLE_NAME}' no encontrado.")
else: df_analysis = globals()[DATASET_VARIABLE_NAME].copy(); print(f"Usando '{DATASET_VARIABLE_NAME}'.")
required_cols_train = ['fund_symbol', 'price_date', 'adj_close'] + REGRESSOR_COLUMNS
missing_cols = [col for col in required_cols_train if col not in df_analysis.columns]
if missing_cols: raise ValueError(f"Faltan cols en {DATASET_VARIABLE_NAME}: {missing_cols}")
try: df_analysis['price_date'] = pd.to_datetime(df_analysis['price_date']); print("'price_date' entreno a datetime.")
except Exception as e: print(f"Error convirtiendo fecha entreno: {e}"); exit()
df_analysis.dropna(subset=['adj_close'], inplace=True)
if REGRESSOR_COLUMNS: # ... (Lógica relleno NaNs regresores entrenamiento) ...
    for col in REGRESSOR_COLUMNS:
        df_analysis[col] = df_analysis.groupby('fund_symbol')[col].ffill().bfill()
        if df_analysis[col].isnull().any(): print(f"WARN: NaNs regresor entreno '{col}'.")


# --- 3. SELECCIONAR TODOS los ETFs ---
print(f"\nIdentificando todos ETFs únicos...")
top_N_etfs_list = df_analysis['fund_symbol'].unique().tolist()
if not top_N_etfs_list: raise ValueError(f"No 'fund_symbol' únicos.");
actual_n_etfs = len(top_N_etfs_list); print(f"Se encontraron {actual_n_etfs} ETFs únicos.")
TOP_N_ETFS = actual_n_etfs; print(f"  (Procesando {TOP_N_ETFS} ETFs)")
print("-" * 50)

# --- 4. BUCLE PRINCIPAL ---
results_prophet = {}
etf_counter = 0
for etf_symbol in top_N_etfs_list:
    etf_counter += 1; print(f"\n{'='*15} Procesando ETF {etf_counter}/{TOP_N_ETFS}: {etf_symbol} {'='*15}")
    results_prophet[etf_symbol] = {'params': {}}; is_log_transformed_etf = False
    # --- 4.1 Prep Datos Entrenamiento ---
    df_etf = df_analysis[df_analysis['fund_symbol'] == etf_symbol].copy().sort_values(by='price_date').reset_index(drop=True)
    cols_to_keep = ['price_date', 'adj_close'] + REGRESSOR_COLUMNS
    df_prophet_train = df_etf[cols_to_keep].rename(columns={'price_date': 'ds', 'adj_close': 'y'}).dropna(subset=['ds', 'y'])
    if USE_LOG_TRANSFORM:
        if (df_prophet_train['y'] <= 0).any(): print(f"WARN: Valores <= 0 {etf_symbol}. Saltando Log.")
        else: df_prophet_train['y'] = np.log(df_prophet_train['y']); is_log_transformed_etf = True; print(f"Log Transform aplicada {etf_symbol}.")
    min_data_points_total = 30; can_run_cv = True
    try: # ... (check datos CV) ...
        min_days_cv = pd.Timedelta(CV_INITIAL).days + pd.Timedelta(CV_HORIZON).days; data_days = (df_prophet_train['ds'].max() - df_prophet_train['ds'].min()).days
        if len(df_prophet_train) < min_data_points_total: raise ValueError(f"Insuficientes ({len(df_prophet_train)})")
        if data_days < min_days_cv: print(f"WARN: Rango ({data_days} días) corto para CV."); can_run_cv = False
    except Exception as e_check: print(f"ERROR check datos {etf_symbol}: {e_check}"); results_prophet[etf_symbol]['error'] = f"Err check: {e_check}"; del df_etf, df_prophet_train; gc.collect(); continue
    last_train_date = df_prophet_train['ds'].max(); results_prophet[etf_symbol]['last_real_date'] = last_train_date
    print(f"Entren. listo. Última fecha: {last_train_date.strftime('%Y-%m-%d')}. Puntos: {len(df_prophet_train)}")

    # --- 4.2 Tuning (Opcional) ---
    best_params_for_etf = PROPHET_BASE_PARAMS.copy(); best_run = None
    if PERFORM_HYPERPARAMETER_TUNING and EJECUTAR_CROSS_VALIDATION and can_run_cv: # ... (lógica tuning igual) ...
        print(f"\n--- Iniciando Tuning {etf_symbol} ---"); param_combinations = [dict(zip(PARAM_GRID.keys(), v)) for v in itertools.product(*PARAM_GRID.values())]; tuning_results = []
        for params_to_try in param_combinations:
            current_trial_params = {**PROPHET_BASE_PARAMS, **params_to_try}
            try: # ... (ejecución CV para tuning) ...
                m_tune=Prophet(**current_trial_params,holidays=CUSTOM_HOLIDAYS);
                if REGRESSOR_COLUMNS: [m_tune.add_regressor(r) for r in REGRESSOR_COLUMNS]
                m_tune.fit(df_prophet_train);
                df_cv_tune=cross_validation(m_tune,initial=CV_INITIAL,period=CV_PERIOD,horizon=CV_HORIZON,parallel="processes",disable_tqdm=True);
                df_p_tune=performance_metrics(df_cv_tune); metric_value=df_p_tune[CV_METRIC_TO_OPTIMIZE].mean();
                tuning_results.append({'params':params_to_try,'metric':metric_value}); del m_tune,df_cv_tune,df_p_tune; gc.collect()
            except Exception as e_tune: print(f"  ERROR CV {params_to_try}: {e_tune}"); tuning_results.append({'params':params_to_try,'metric':float('inf')})
        if tuning_results: # ... (selección mejores params) ...
            best_run=min(tuning_results,key=lambda x: x['metric']);
            if best_run['metric']!=float('inf'): best_params_for_etf.update(best_run['params']); print(f"\nMejores params {etf_symbol}: {best_run['params']} ({CV_METRIC_TO_OPTIMIZE}={best_run['metric']:.4f})")
            else: print("\nTuning falló.")
        else: print("\nNo resultados tuning.")
    else: # Mensajes omisión
        if not (PERFORM_HYPERPARAMETER_TUNING and EJECUTAR_CROSS_VALIDATION): print("\nTuning OMITIDO (desactivado).")
        elif not can_run_cv: print("\nTuning OMITIDO (datos insuficientes).")
    results_prophet[etf_symbol]['params'] = best_params_for_etf.copy(); print(f"Params finales {etf_symbol}: {best_params_for_etf}")

    # --- 4.3 Modelo Final ---
    print(f"\nEntrenando modelo FINAL {etf_symbol}...")
    model_final = Prophet(**best_params_for_etf, holidays=CUSTOM_HOLIDAYS)
    if REGRESSOR_COLUMNS: print(f"Añadiendo regresores: {REGRESSOR_COLUMNS}"); [model_final.add_regressor(r) for r in REGRESSOR_COLUMNS]
    try:
        model_final.fit(df_prophet_train); print("Entrenamiento final completado.")
    except Exception as e_fit_final:
        # Bloque except con indentación corregida
        print(f"ERROR entrenando FINAL: {e_fit_final}")
        results_prophet[etf_symbol]['error'] = f"Err entreno final: {e_fit_final}"
        if 'df_etf' in locals():
            del df_etf
        if 'df_prophet_train' in locals():
            del df_prophet_train
        gc.collect();
        if 'model_final' in locals():
            del model_final
        continue # Saltar al siguiente ETF

    # --- 4.4 Resumen CV/Tuning ---
    # ... (lógica igual) ...
    if best_run and best_run['metric'] != float('inf') and PERFORM_HYPERPARAMETER_TUNING: results_prophet[etf_symbol]['cv_metrics_summary'] = f"Mejor {CV_METRIC_TO_OPTIMIZE} Tuning: {best_run['metric']:.4f}"
    elif not can_run_cv: results_prophet[etf_symbol]['cv_metrics_summary'] = "CV Omitida (datos insuf.)"
    elif not EJECUTAR_CROSS_VALIDATION: results_prophet[etf_symbol]['cv_metrics_summary'] = "CV Omitida (desactivada)"
    else: results_prophet[etf_symbol]['cv_metrics_summary'] = "CV no ejecutada (sin tuning)"


    # --- 4.5 Forecast Final ---
    print(f"\nGenerando Forecast {etf_symbol}..."); forecast_raw = None; future_df_final = None; forecast_final_scale = None
    try: # ... (lógica forecast y reversión log igual) ...
        target_end_date = pd.to_datetime(FUTURE_END_DATE); freq = 'B'; future_dates = pd.date_range(start=last_train_date + pd.Timedelta(days=1), end=target_end_date, freq=freq)
        future_df_base = pd.DataFrame({'ds': future_dates}); future_df_final = future_df_base.copy()
        if REGRESSOR_COLUMNS: # ... (lógica añadir regresores futuros) ...
             print("Añadiendo regresores futuros...");
             try: # ... (igual que antes) ...
                 df_actual_full=pd.read_csv(ACTUAL_DATA_FILE); df_actual_full['ds']=pd.to_datetime(df_actual_full['price_date'])
                 df_reg_fut=df_actual_full.loc[df_actual_full['fund_symbol'] == etf_symbol, ['ds'] + REGRESSOR_COLUMNS].copy()
                 future_df_final=pd.merge(future_df_final, df_reg_fut, on='ds', how='left')
                 for col in REGRESSOR_COLUMNS:
                     future_df_final[col]=future_df_final[col].ffill().bfill()
                     if future_df_final[col].isnull().any(): last_known = df_prophet_train[col].iloc[-1]; future_df_final[col].fillna(last_known, inplace=True); print(f"WARN: NaNs reg futuro '{col}' rellenados.")
                 del df_actual_full, df_reg_fut
             except Exception as e_reg_fut: print(f"ERROR obteniendo regresores futuros: {e_reg_fut}"); raise
        print("Realizando predicción..."); future_df_final = future_df_final[future_df_final['ds'] <= target_end_date];
        if future_df_final.empty: raise ValueError("Future DF vacío.")
        forecast_raw = model_final.predict(future_df_final); print("Predicción completada.")
        forecast_final_scale = forecast_raw.copy()
        if is_log_transformed_etf: # Revertir log
             cols_to_exp=['yhat', 'yhat_lower', 'yhat_upper','trend', 'trend_lower', 'trend_upper']
             for col in cols_to_exp:
                 if col in forecast_final_scale.columns:
                     try: forecast_final_scale[col]=np.exp(forecast_final_scale[col])
                     except Exception as e_exp: print(f"ERROR np.exp '{col}': {e_exp}")
             results_prophet[etf_symbol]['log_transform_applied'] = True
        else: results_prophet[etf_symbol]['log_transform_applied'] = False
        print("Guardando forecast (escala original)...")
        # Guardar SOLO ds y yhat para frontend
        cols_save=['ds', 'yhat']; cols_pres=[c for c in cols_save if c in forecast_final_scale.columns]
        if 'ds' in cols_pres and 'yhat' in cols_pres:
            results_prophet[etf_symbol]['forecast'] = forecast_final_scale[cols_pres].copy() # Guardar DF procesado
            print(f"Forecast final ('ds', 'yhat') guardado. Última fecha: {forecast_final_scale['ds'].max().strftime('%Y-%m-%d')}.")
        else:
             print("ERROR: Columnas 'ds' o 'yhat' faltan. No guardado."); results_prophet[etf_symbol]['error_forecast'] = "Faltan cols ds/yhat."

    except Exception as e_pred: # ... (manejo error forecast igual, con indentación corregida) ...
        print(f"ERROR generando forecast final {etf_symbol}: {e_pred}"); results_prophet[etf_symbol]['error_forecast'] = str(e_pred); traceback.print_exc();
        if 'model_final' in locals(): del model_final
        if 'df_etf' in locals(): del df_etf
        if 'df_prophet_train' in locals(): del df_prophet_train
        if 'future_df_final' in locals(): del future_df_final
        if 'forecast_raw' in locals(): del forecast_raw # Cambiado de forecast_log_scale
        if 'forecast_final_scale' in locals(): del forecast_final_scale
        gc.collect(); continue

    # --- 4.7 Limpieza ---
    print(f"Limpiando memoria {etf_symbol}...");
    if 'model_final' in locals(): del model_final
    if 'df_etf' in locals(): del df_etf
    if 'df_prophet_train' in locals(): del df_prophet_train
    if 'future_df_final' in locals(): del future_df_final
    if 'forecast_raw' in locals(): del forecast_raw # Cambiado de forecast_log_scale
    if 'forecast_final_scale' in locals(): del forecast_final_scale
    if 'best_run' in locals(): del best_run
    if 'tuning_results' in locals(): del tuning_results
    gc.collect(); print("Limpieza completada.")
    print(f"\n{'='*15} Procesamiento {etf_symbol} completado {'='*15}"); print("-" * 70)


# --- 5. RESUMEN ANÁLISIS (Opcional)---
# print("\n--- Resumen Resultados Análisis ---")
# # ... (código resumen) ...
# print("\n--- Fin Resumen Análisis ---")

# --- 6. COMPARACIÓN CON DATOS REALES (Opcional) ---
# Comentar si no se necesita el plot de matplotlib aquí
# print("\n\n###########################################################")
# print(f"      COMPARACIÓN DATOS REALES DESDE '{ACTUAL_DATA_FILE}'")
# # ... (código completo Sección 6 para plots) ...
# print("\n--- Fin Comparación ---")


# --- 7. GUARDAR RESULTADOS PARA EL FRONT-END ---
print("\n\n###########################################################")
print(f"      GUARDANDO RESULTADOS PARA FRONT-END en '{FRONTEND_DATA_FILE}'")
print("###########################################################")
results_to_save = {}
if 'results_prophet' in locals() and isinstance(results_prophet, dict):
    print(f"Procesando {len(results_prophet)} ETFs para guardar...")
    for etf, data in results_prophet.items():
        if ('forecast' in data and isinstance(data['forecast'], pd.DataFrame) and
            not data['forecast'].empty and 'ds' in data['forecast'].columns and
            'yhat' in data['forecast'].columns):
            try: # ... (lógica guardar igual) ...
                forecast_df = data['forecast'][['ds', 'yhat']].copy(); forecast_df['ds'] = pd.to_datetime(forecast_df['ds']); forecast_df.dropna(subset=['yhat'], inplace=True)
                if not forecast_df.empty: results_to_save[etf] = {'forecast': forecast_df}; print(f"  - Forecast {etf} prep ({len(forecast_df)} filas).")
                else: print(f"  - WARN: Forecast {etf} vacío post-limpieza.")
            except Exception as e_prep_save: print(f"  - ERROR prep forecast {etf}: {e_prep_save}.")
        else: print(f"  - WARN: No se guardará {etf} (sin forecast válido).")
    if results_to_save:
        try:
            with open(FRONTEND_DATA_FILE, 'wb') as f: pickle.dump(results_to_save, f)
            print(f"\nResultados ({len(results_to_save)} ETFs) guardados en: '{FRONTEND_DATA_FILE}'")
            print(f"  ETFs guardados: {list(results_to_save.keys())}")
        except Exception as e_save: print(f"\nERROR guardando: {e_save}"); traceback.print_exc()
    else: print("\nNo se encontraron resultados válidos para guardar.")
else: print("\nVariable 'results_prophet' no encontrada.")
print("\n--- Fin Guardado para Front-End ---")

print("\n--- Script de Análisis Finalizado ---")


--- Iniciando Análisis Prophet ---
Usando 'df_top20'.
'price_date' entreno a datetime.

Identificando todos ETFs únicos...
Se encontraron 20 ETFs únicos.
  (Procesando 20 ETFs)
--------------------------------------------------

=============== Procesando ETF 1/20: SPY ===============
Log Transform aplicada SPY.
Entren. listo. Última fecha: 2021-11-30. Puntos: 7263

Tuning OMITIDO (desactivado).
Params finales SPY: {'seasonality_mode': 'multiplicative', 'daily_seasonality': False, 'weekly_seasonality': True, 'yearly_seasonality': True, 'interval_width': 0.8}

Entrenando modelo FINAL SPY...


DEBUG:cmdstanpy:input tempfile: /tmp/tmp9pmiwdvw/jgtk0xvy.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9pmiwdvw/qz81gzyh.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=95332', 'data', 'file=/tmp/tmp9pmiwdvw/jgtk0xvy.json', 'init=/tmp/tmp9pmiwdvw/qz81gzyh.json', 'output', 'file=/tmp/tmp9pmiwdvw/prophet_model9e_hq7qy/prophet_model-20250515171342.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:13:42 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:14:05 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Entrenamiento final completado.

Generando Forecast SPY...
Realizando predicción...


DEBUG:cmdstanpy:input tempfile: /tmp/tmp9pmiwdvw/us73w_tg.json


Predicción completada.
Guardando forecast (escala original)...
Forecast final ('ds', 'yhat') guardado. Última fecha: 2030-01-01.
Limpiando memoria SPY...
Limpieza completada.

=============== Procesamiento SPY completado ===============
----------------------------------------------------------------------

=============== Procesando ETF 2/20: QLD ===============
Log Transform aplicada QLD.
Entren. listo. Última fecha: 2021-11-30. Puntos: 3889

Tuning OMITIDO (desactivado).
Params finales QLD: {'seasonality_mode': 'multiplicative', 'daily_seasonality': False, 'weekly_seasonality': True, 'yearly_seasonality': True, 'interval_width': 0.8}

Entrenando modelo FINAL QLD...


...

Entrenamiento final completado.

Generando Forecast FRAK...
Realizando predicción...


DEBUG:cmdstanpy:input tempfile: /tmp/tmp9pmiwdvw/w8k2lowl.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp9pmiwdvw/kxl9t74y.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=89332', 'data', 'file=/tmp/tmp9pmiwdvw/w8k2lowl.json', 'init=/tmp/tmp9pmiwdvw/kxl9t74y.json', 'output', 'file=/tmp/tmp9pmiwdvw/prophet_model08t1ioz3/prophet_model-20250515171542.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:15:42 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


Predicción completada.
Guardando forecast (escala original)...
Forecast final ('ds', 'yhat') guardado. Última fecha: 2030-01-01.
Limpiando memoria FRAK...
Limpieza completada.

=============== Procesamiento FRAK completado ===============
----------------------------------------------------------------------

=============== Procesando ETF 20/20: AMER ===============
Log Transform aplicada AMER.
WARN: Rango (410 días) corto para CV.
Entren. listo. Última fecha: 2021-11-30. Puntos: 283

Tuning OMITIDO (desactivado).
Params finales AMER: {'seasonality_mode': 'multiplicative', 'daily_seasonality': False, 'weekly_seasonality': True, 'yearly_seasonality': True, 'interval_width': 0.8}

Entrenando modelo FINAL AMER...


17:15:42 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Entrenamiento final completado.

Generando Forecast AMER...
Realizando predicción...
Predicción completada.
Guardando forecast (escala original)...
Forecast final ('ds', 'yhat') guardado. Última fecha: 2030-01-01.
Limpiando memoria AMER...
Limpieza completada.

=============== Procesamiento AMER completado ===============
----------------------------------------------------------------------


###########################################################
      GUARDANDO RESULTADOS PARA FRONT-END en 'prophet_frontend_data_v2.pkl'
###########################################################
Procesando 20 ETFs para guardar...
  - Forecast SPY prep (2110 filas).
  - Forecast QLD prep (2110 filas).
  - Forecast RXL prep (2110 filas).
  - Forecast USD prep (2110 filas).
  - Forecast ROM prep (2110 filas).
  - Forecast UCC prep (2110 filas).
  - Forecast TNA prep (2110 filas).
  - Forecast TECL prep (2110 filas).
  - Forecast MIDU prep (2110 filas).
  - Forecast UPRO prep (2110 filas).
  - Forec

In [5]:
# --- Definición del Código de la Aplicación Streamlit ---
# Definir el nombre del archivo .pkl que se espera (DEBE COINCIDIR con Celda 2)
FRONTEND_DATA_FILE_APP = 'prophet_frontend_data_v2.pkl' # Mismo nombre usado en Celda 2 y 3

streamlit_app_code = f"""
# -*- coding: utf-8 -*-
# calculadora_etf_app.py (basado en retornos predichos)

import streamlit as st
import pandas as pd
import numpy as np
import pickle
from datetime import timedelta
import plotly.graph_objects as go
import os
import traceback # Importar traceback para errores más detallados

# <<< DEFINICIÓN VARIABLE GLOBAL DENTRO DEL SCRIPT >>>
FRONTEND_DATA_FILE_APP = '{FRONTEND_DATA_FILE_APP}' # Usa el valor de la variable externa

# Config Página
st.set_page_config(page_title="Calculadora Compuesta ETF", page_icon="📈", layout="wide")

# Carga Datos (con cache y validación)
@st.cache_data
def load_results(filepath): # <<< CORREGIDO: Sin duplicación >>>
    st.write(f"Intentando cargar datos desde: {{filepath}}")
    if not os.path.exists(filepath):
        st.error(f"ERROR CRÍTICO: Archivo '{{filepath}}' no encontrado.")
        st.info("Ejecuta el script de análisis para generarlo.")
        try: st.info(f"Archivos directorio actual: {{os.listdir('.')}}")
        except Exception: pass
        return None
    try:
        with open(filepath, 'rb') as f: results = pickle.load(f)
        st.write("Archivo .pkl cargado.")
        if not isinstance(results, dict) or not results:
            st.error("Error: Archivo .pkl vacío o formato incorrecto."); return None
        valid_etfs = {{}}
        st.write(f"Validando {{len(results)}} ETFs del archivo...")
        for etf, data in results.items():
            valid = False
            if (isinstance(data, dict) and 'forecast' in data and
                isinstance(data['forecast'], pd.DataFrame) and not data['forecast'].empty and
                'ds' in data['forecast'].columns and 'yhat' in data['forecast'].columns):
                try:
                    data['forecast']['ds'] = pd.to_datetime(data['forecast']['ds'])
                    data['forecast'].dropna(subset=['yhat'], inplace=True)
                    if not data['forecast'].empty: valid_etfs[etf] = data; valid = True
                except Exception as e_dt: st.warning(f"Error formato/datos {{etf}}: {{e_dt}}.")
            if not valid: st.warning(f"Datos inválidos/incompletos {{etf}}.")
        if not valid_etfs: st.error("No se encontraron ETFs válidos."); return None
        st.success(f"Carga completada. {{len(valid_etfs)}} ETFs disponibles.")
        return valid_etfs # Correctamente indentado
    except Exception as e: st.error(f"Error inesperado cargando: {{e}}"); st.text(traceback.format_exc()); return None

# Ejecución Principal App
prophet_results = load_results(filepath=FRONTEND_DATA_FILE_APP) # Pasa el filepath definido arriba

st.title("📈 Calculadora de Interés Compuesto con Predicciones Prophet")

if prophet_results is None:
    st.subheader("Error al cargar datos")
    st.warning("No se pudieron cargar datos de predicción. Verifica el archivo .pkl.")
    st.stop()

# --- UI Principal (si los datos cargaron bien) ---
# ... (El resto del código UI y lógica de cálculo permanece igual que antes) ...
st.markdown("Simula el crecimiento de una inversión usando los **retornos predichos**.")
available_etfs = sorted(list(prophet_results.keys()))
if not available_etfs: st.error("No hay ETFs disponibles."); st.stop()
col1, col2, col3 = st.columns(3)
with col1:
    selected_etf = st.selectbox("1. Selecciona ETF:", available_etfs, index=0)
    initial_investment = st.number_input("2. Inversión Inicial (€/$):", min_value=0.0, value=1000.0, step=100.0, format="%.2f")
with col2:
    monthly_contribution = st.number_input("3. Aportación Mensual (€/$):", min_value=0.0, value=100.0, step=50.0, format="%.2f")
    try: # Lógica max_years_input
        max_date = max(d['forecast']['ds'].max() for d in prophet_results.values())
        min_date = min(d['forecast']['ds'].min() for d in prophet_results.values())
        max_years_poss = int((max_date - min_date).days / 365); max_yrs_in = min(max_years_poss, 15)
        if max_yrs_in < 1: max_yrs_in = 1
    except: max_yrs_in = 10
    investment_years = st.number_input(f"4. Duración (Años - Máx. {{max_yrs_in}}):", min_value=1, max_value=max_yrs_in, value=min(3, max_yrs_in), step=1)
with col3: st.markdown("<br/><br/>", unsafe_allow_html=True); calculate_button = st.button("🚀 Calcular Proyección", use_container_width=True)
st.divider()
if calculate_button and selected_etf:
    st.header(f"Proyección para {{selected_etf}}")
    try:
        forecast_data = prophet_results[selected_etf]['forecast'].copy().sort_values('ds').set_index('ds')
        start_date = forecast_data.index.min(); end_date_sim = start_date + pd.DateOffset(years=investment_years); max_forecast_date = forecast_data.index.max()
        if end_date_sim > max_forecast_date: st.warning(f"Limitando simulación a {{max_forecast_date.strftime('%Y-%m-%d')}}."); end_date_sim = max_forecast_date;
        sim_forecast = forecast_data[(forecast_data.index >= start_date) & (forecast_data.index <= end_date_sim)].copy()
        if sim_forecast.empty: st.error(f"No hay predicciones para {{selected_etf}} en período."); st.stop()
        monthly_prices_start = sim_forecast['yhat'].resample('MS').first(); monthly_returns = monthly_prices_start.pct_change().fillna(0)
        current_balance = initial_investment; investment_history = pd.DataFrame(columns=['Balance', 'Contribuciones', 'Crecimiento_Mes'])
        record_date_initial = start_date.to_period('M').start_time - timedelta(days=1)
        investment_history.loc[record_date_initial] = [initial_investment, initial_investment, 0.0]
        for month_start_date, monthly_return in monthly_returns.items():
            balance_before_contrib = current_balance; growth_this_month = 0
            if month_start_date != monthly_returns.index.min(): growth_this_month = balance_before_contrib * monthly_return; current_balance += growth_this_month
            if month_start_date > record_date_initial: current_balance += monthly_contribution; total_contrib_so_far = investment_history['Contribuciones'].iloc[-1] + monthly_contribution
            else: total_contrib_so_far = initial_investment
            investment_history.loc[month_start_date] = [current_balance, total_contrib_so_far, growth_this_month]
        final_balance = investment_history['Balance'].iloc[-1]; total_contributions = investment_history['Contribuciones'].iloc[-1]; total_growth = final_balance - total_contributions
        st.subheader("Resultados Simulación")
        col_res1, col_res2, col_res3 = st.columns(3)
        with col_res1: st.metric("Valor Final Proyectado", f"{{final_balance:,.2f}} €/$")
        with col_res2: st.metric("Aportaciones Totales", f"{{total_contributions:,.2f}} €/$")
        with col_res3: st.metric("Crecimiento Estimado", f"{{total_growth:,.2f}} €/$", delta=f"{{total_growth/total_contributions*100 if total_contributions>0 else 0:.1f}}%")
        st.subheader("Gráfico Crecimiento Proyectado")
        investment_history.index = pd.to_datetime(investment_history.index)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=investment_history.index, y=investment_history['Balance'], mode='lines', name='Valor Inversión', line=dict(color='green', width=2), hovertemplate='<b>Fecha</b>: %{{x|%Y-%m-%d}}<br><b>Balance</b>: %{{y:,.2f}} €/$<extra></extra>'))
        fig.update_layout(title=f'Crecimiento Inversión {{selected_etf}} (Predicción)', xaxis_title='Fecha', yaxis_title='Valor (€/$)', xaxis_tickformat='%Y-%m-%d', hovermode='x unified', legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
        st.plotly_chart(fig, use_container_width=True)
    except Exception as e_calc: st.error(f"Error cálculo/gráficación: {{e_calc}}"); st.text(traceback.format_exc())
# Footer
st.sidebar.markdown("---"); st.sidebar.info("Calculadora ilustrativa (predicciones Prophet). No garantía.")

"""

# --- Nombre del archivo para guardar el código Streamlit ---
streamlit_app_filename = "calculadora_etf_app.py"

# --- Guardar el código Streamlit en un archivo .py ---
try:
    with open(streamlit_app_filename, "w", encoding="utf-8") as f: f.write(streamlit_app_code)
    print(f"\nCódigo de la aplicación Streamlit guardado en '{streamlit_app_filename}'")
except Exception as e_write: print(f"\nERROR al guardar el script de Streamlit: {e_write}")


Código de la aplicación Streamlit guardado en 'calculadora_etf_app.py'


In [6]:
# --- Configuración e Inicio de ngrok ---

# 1. !!! REEMPLAZA CON TU AUTHTOKEN REAL de ngrok.com !!!
NGROK_AUTH_TOKEN = "2w5gkPnuxS2MaHakM0sGPAMNMdu_2dnPu3XzCXhSh1nQWin4j"

# 2. Configurar ngrok
print("\nConfigurando ngrok...")
!ngrok authtoken {NGROK_AUTH_TOKEN}

# 3. Importar pyngrok
from pyngrok import ngrok
import time

# 4. Matar túneles previos
print("Cerrando túneles ngrok existentes...")
ngrok.kill()
time.sleep(2)

# 5. Iniciar Streamlit en segundo plano (usa el nombre de archivo correcto)
print(f"\nIniciando Streamlit desde '{streamlit_app_filename}' en segundo plano...")
!streamlit run {streamlit_app_filename} &>/dev/null& # Oculta salida normal
print("Esperando unos segundos a que Streamlit arranque...")
time.sleep(12) # Dar un poco más de tiempo

# 6. Crear túnel ngrok
port = 8501
print(f"\nCreando túnel ngrok hacia el puerto {port}...")
try:
    public_url = ngrok.connect(port)
    print("*"*50); print(f"Aplicación Streamlit lista."); print(f"Accede en URL pública:"); print(public_url); print("*"*50)
except Exception as e_ngrok:
    print(f"ERROR al iniciar ngrok: {e_ngrok}"); print("Verifica Authtoken.")

# Mantener celda ejecutándose


Configurando ngrok...
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Cerrando túneles ngrok existentes...

Iniciando Streamlit desde 'calculadora_etf_app.py' en segundo plano...
Esperando unos segundos a que Streamlit arranque...

Creando túnel ngrok hacia el puerto 8501...
**************************************************
Aplicación Streamlit lista.
Accede en URL pública:
NgrokTunnel: "https://1cf9-34-106-243-173.ngrok-free.app" -> "http://localhost:8501"
**************************************************


# IDEA B

In [ ]:
# -*- coding: utf-8 -*-
"""
Script Completo para Análisis y Comparación de ETFs con Prophet.

Versión Final v5.2 (Incluye Guardado para Frontend y Corrección SyntaxError):
- Selección por Número de Datos.
- Transformación Logarítmica Opcional (con manejo robusto de reversión).
- Regresores Externos Opcionales.
- Ajuste de Hiperparámetros Opcional (vía CV).
- Comparación Final con Datos Reales y Plot Manual (EJE Y LOGARÍTMICO).
- *** Corrección SyntaxError en limpieza de memoria (Sección 4.7). ***
- Sección 7 para guardar resultados para Streamlit app.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import gc # Garbage collector
import itertools # Para el grid search
import traceback # Para errores detallados
import pickle # Para guardar resultados
import os # Para listar archivos (depuración opcional)

# Instala Prophet si no lo tienes: pip install prophet cmdstanpy
try:
    from prophet import Prophet
    from prophet.diagnostics import cross_validation, performance_metrics
except ImportError:
    print("*"*30); print("ERROR: 'prophet' no instalado. Ejecuta: pip install prophet cmdstanpy"); print("*"*30); exit()

# Ignorar advertencias
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)


# --- 1. CONFIGURACIÓN GENERAL ---
TOP_N_ETFS = 5
DATASET_VARIABLE_NAME = 'df_top20' # <<< Asegúrate que esta variable existe en tu entorno Colab
FUTURE_END_DATE = '2025-03-10'
ACTUAL_DATA_FILE = 'ETF_cohortes.csv'

# --- 1.1 CONFIGURACIÓN DE MEJORAS ---
USE_LOG_TRANSFORM = True
REGRESSOR_COLUMNS = []
PERFORM_HYPERPARAMETER_TUNING = True # Cambiar a False para saltar tuning
PARAM_GRID = { 'changepoint_prior_scale': [0.01, 0.1, 0.5], 'seasonality_prior_scale': [1.0, 10.0, 20.0], 'holidays_prior_scale': [1.0, 10.0, 20.0] }
PROPHET_BASE_PARAMS = { 'seasonality_mode': 'multiplicative', 'daily_seasonality': False, 'weekly_seasonality': True, 'yearly_seasonality': True, 'interval_width': 0.80 }
CUSTOM_HOLIDAYS = None

# --- 1.2 CONFIGURACIÓN CV ---
EJECUTAR_CROSS_VALIDATION = True # Cambiar a False para saltar CV y Tuning
CV_INITIAL = '730 days'; CV_PERIOD = '90 days'; CV_HORIZON = '180 days'; CV_METRIC_TO_OPTIMIZE = 'rmse'


# --- 2. CARGA Y PREPARACIÓN DATAFRAME ENTRENAMIENTO ---
if DATASET_VARIABLE_NAME not in globals(): raise NameError(f"'{DATASET_VARIABLE_NAME}' no encontrado.")
else: df_analysis = globals()[DATASET_VARIABLE_NAME].copy(); print(f"Usando '{DATASET_VARIABLE_NAME}'.")
required_cols_train = ['fund_symbol', 'price_date', 'adj_close'] + REGRESSOR_COLUMNS
missing_cols = [col for col in required_cols_train if col not in df_analysis.columns]
if missing_cols: raise ValueError(f"Faltan cols en {DATASET_VARIABLE_NAME}: {missing_cols}")
try: df_analysis['price_date'] = pd.to_datetime(df_analysis['price_date']); print("'price_date' entrenamiento a datetime.")
except Exception as e: print(f"Error convirtiendo fecha entreno: {e}"); exit()
df_analysis.dropna(subset=['adj_close'], inplace=True)
if REGRESSOR_COLUMNS:
    for col in REGRESSOR_COLUMNS:
        df_analysis[col] = df_analysis.groupby('fund_symbol')[col].ffill().bfill()
        if df_analysis[col].isnull().any(): print(f"WARN: NaNs regresor entreno '{col}'.")

# --- 3. SELECCIONAR TOP N ETFs POR NÚMERO DE DATOS ---
print(f"\nIdentificando {TOP_N_ETFS} ETFs con más datos...")
etf_data_counts = df_analysis.groupby('fund_symbol').size().reset_index(name='data_count')
etf_data_counts_sorted = etf_data_counts.sort_values(by='data_count', ascending=False)
top_N_etfs_list = etf_data_counts_sorted['fund_symbol'].head(TOP_N_ETFS).tolist()
if not top_N_etfs_list: raise ValueError("No se pudo contar datos.");
if len(top_N_etfs_list) < TOP_N_ETFS: print(f"Warn: Solo {len(top_N_etfs_list)} ETFs."); TOP_N_ETFS = len(top_N_etfs_list)
print(f"Se procesarán {TOP_N_ETFS} ETFs (más datos):")
selected_counts_info = etf_data_counts_sorted.head(TOP_N_ETFS)
for i, (index, row) in enumerate(selected_counts_info.iterrows()): print(f"  {i+1}. {row['fund_symbol']} ({row['data_count']} puntos)")
print("-" * 50)

# --- 4. BUCLE PRINCIPAL PARA PROCESAR CADA ETF ---
results_prophet = {}
for etf_symbol in top_N_etfs_list:
    print(f"\n{'='*15} Procesando ETF: {etf_symbol} {'='*15}")
    results_prophet[etf_symbol] = {'params': {}}; is_log_transformed_etf = False
    # --- 4.1 Prep Datos Entrenamiento ---
    df_etf = df_analysis[df_analysis['fund_symbol'] == etf_symbol].copy().sort_values(by='price_date').reset_index(drop=True)
    cols_to_keep = ['price_date', 'adj_close'] + REGRESSOR_COLUMNS
    df_prophet_train = df_etf[cols_to_keep].rename(columns={'price_date': 'ds', 'adj_close': 'y'}).dropna(subset=['ds', 'y'])
    if USE_LOG_TRANSFORM:
        if (df_prophet_train['y'] <= 0).any(): print(f"WARN: Valores <= 0 {etf_symbol}. Saltando Log.")
        else: df_prophet_train['y'] = np.log(df_prophet_train['y']); is_log_transformed_etf = True; print(f"Log Transform aplicada {etf_symbol}.")
    min_data_points_total = 30; can_run_cv = True
    try:
        min_days_cv = pd.Timedelta(CV_INITIAL).days + pd.Timedelta(CV_HORIZON).days; data_days = (df_prophet_train['ds'].max() - df_prophet_train['ds'].min()).days
        if len(df_prophet_train) < min_data_points_total: raise ValueError(f"Insuficientes ({len(df_prophet_train)})")
        if data_days < min_days_cv: print(f"WARN: Rango ({data_days} días) corto para CV."); can_run_cv = False
    except Exception as e_check: print(f"ERROR check datos {etf_symbol}: {e_check}"); results_prophet[etf_symbol]['error'] = f"Err check: {e_check}"; del df_etf, df_prophet_train; gc.collect(); continue
    last_train_date = df_prophet_train['ds'].max(); results_prophet[etf_symbol]['last_real_date'] = last_train_date
    print(f"Entren. listo. Última fecha: {last_train_date.strftime('%Y-%m-%d')}. Puntos: {len(df_prophet_train)}")

    # --- 4.2 Tuning ---
    best_params_for_etf = PROPHET_BASE_PARAMS.copy(); best_run = None
    if PERFORM_HYPERPARAMETER_TUNING and EJECUTAR_CROSS_VALIDATION and can_run_cv:
        print(f"\n--- Iniciando Tuning {etf_symbol} ---")
        param_combinations = [dict(zip(PARAM_GRID.keys(), v)) for v in itertools.product(*PARAM_GRID.values())]; tuning_results = []
        for params_to_try in param_combinations:
            current_trial_params = {**PROPHET_BASE_PARAMS, **params_to_try}
            try:
                m_tune = Prophet(**current_trial_params, holidays=CUSTOM_HOLIDAYS)
                if REGRESSOR_COLUMNS: [m_tune.add_regressor(r) for r in REGRESSOR_COLUMNS]
                m_tune.fit(df_prophet_train)
                df_cv_tune = cross_validation(m_tune, initial=CV_INITIAL, period=CV_PERIOD, horizon=CV_HORIZON, parallel="processes", disable_tqdm=True)
                df_p_tune = performance_metrics(df_cv_tune); metric_value = df_p_tune[CV_METRIC_TO_OPTIMIZE].mean()
                tuning_results.append({'params': params_to_try, 'metric': metric_value})
                del m_tune, df_cv_tune, df_p_tune; gc.collect()
            except Exception as e_tune: print(f"  ERROR CV {params_to_try}: {e_tune}"); tuning_results.append({'params': params_to_try, 'metric': float('inf')})
        if tuning_results:
            best_run = min(tuning_results, key=lambda x: x['metric'])
            if best_run['metric'] != float('inf'): best_params_for_etf.update(best_run['params']); print(f"\nMejores params {etf_symbol}: {best_run['params']} ({CV_METRIC_TO_OPTIMIZE}={best_run['metric']:.4f})")
            else: print("\nTuning falló. Usando base.")
        else: print("\nNo resultados tuning. Usando base.")
    else: # Mensajes de omisión
        if not (PERFORM_HYPERPARAMETER_TUNING and EJECUTAR_CROSS_VALIDATION): print("\nTuning OMITIDO (desactivado).")
        elif not can_run_cv: print("\nTuning OMITIDO (datos insuficientes).")
    results_prophet[etf_symbol]['params'] = best_params_for_etf.copy(); print(f"Params finales {etf_symbol}: {best_params_for_etf}")

    # --- 4.3 Modelo Final ---
    print(f"\nEntrenando modelo FINAL {etf_symbol}...")
    model_final = Prophet(**best_params_for_etf, holidays=CUSTOM_HOLIDAYS)
    if REGRESSOR_COLUMNS: print(f"Añadiendo regresores: {REGRESSOR_COLUMNS}"); [model_final.add_regressor(r) for r in REGRESSOR_COLUMNS]
    try:
        model_final.fit(df_prophet_train); print("Entrenamiento final completado.")
    except Exception as e_fit_final:
        print(f"ERROR entrenando FINAL: {e_fit_final}")
        results_prophet[etf_symbol]['error'] = f"Err entreno final: {e_fit_final}"
        if 'df_etf' in locals(): del df_etf
        if 'df_prophet_train' in locals(): del df_prophet_train
        gc.collect();
        if 'model_final' in locals(): del model_final
        continue

    # --- 4.4 Resumen CV/Tuning ---
    if best_run and best_run['metric'] != float('inf') and PERFORM_HYPERPARAMETER_TUNING: results_prophet[etf_symbol]['cv_metrics_summary'] = f"Mejor {CV_METRIC_TO_OPTIMIZE} Tuning: {best_run['metric']:.4f}"
    elif not can_run_cv: results_prophet[etf_symbol]['cv_metrics_summary'] = "CV Omitida (datos insuf.)"
    elif not EJECUTAR_CROSS_VALIDATION: results_prophet[etf_symbol]['cv_metrics_summary'] = "CV Omitida (desactivada)"
    else: results_prophet[etf_symbol]['cv_metrics_summary'] = "CV no ejecutada (sin tuning)"

    # --- 4.5 Forecast Final ---
    print(f"\nGenerando Forecast {etf_symbol}..."); forecast_log_scale = None; future_df_final = None; forecast_final_scale = None
    try:
        target_end_date = pd.to_datetime(FUTURE_END_DATE); freq = 'B'
        future_dates = pd.date_range(start=last_train_date + pd.Timedelta(days=1), end=target_end_date, freq=freq)
        future_df_base = pd.DataFrame({'ds': future_dates}); future_df_final = future_df_base.copy()
        if REGRESSOR_COLUMNS: # ... (lógica regresores futuros) ...
            print("Añadiendo regresores futuros...");
            try: # ... (igual que antes) ...
                df_actual_full=pd.read_csv(ACTUAL_DATA_FILE); df_actual_full['ds']=pd.to_datetime(df_actual_full['price_date'])
                df_reg_fut=df_actual_full.loc[df_actual_full['fund_symbol'] == etf_symbol, ['ds'] + REGRESSOR_COLUMNS].copy()
                future_df_final=pd.merge(future_df_final, df_reg_fut, on='ds', how='left')
                for col in REGRESSOR_COLUMNS:
                    future_df_final[col]=future_df_final[col].ffill().bfill()
                    if future_df_final[col].isnull().any(): last_known = df_prophet_train[col].iloc[-1]; future_df_final[col].fillna(last_known, inplace=True); print(f"WARN: NaNs reg futuro '{col}' rellenados.")
                del df_actual_full, df_reg_fut
            except Exception as e_reg_fut: print(f"ERROR obteniendo regresores futuros: {e_reg_fut}"); raise
        print("Realizando predicción..."); future_df_final = future_df_final[future_df_final['ds'] <= target_end_date];
        if future_df_final.empty: raise ValueError("Future DF vacío.")
        forecast_log_scale = model_final.predict(future_df_final); print("Predicción completada.")
        #print(f"DEBUG: forecast_log_scale tail (yhat PEQUEÑO si log):\n{forecast_log_scale[['ds', 'yhat']].tail()}")
        forecast_final_scale = forecast_log_scale.copy()
        if is_log_transformed_etf:
            #print(f"DEBUG: INTENTANDO REVERTIR Log {etf_symbol}. Flag={is_log_transformed_etf}")
            cols_to_exp=['yhat', 'yhat_lower', 'yhat_upper','trend', 'trend_lower', 'trend_upper']
            cols_rev = []
            for col in cols_to_exp:
                if col in forecast_final_scale.columns:
                    try: forecast_final_scale[col]=np.exp(forecast_final_scale[col]); cols_rev.append(col)
                    except Exception as e_exp: print(f"ERROR np.exp '{col}': {e_exp}")
            #print(f"DEBUG: Columnas revertidas: {cols_rev}")
            #print(f"DEBUG: forecast_final_scale tail (yhat GRANDE ahora):\n{forecast_final_scale[['ds', 'yhat']].tail()}")
            results_prophet[etf_symbol]['log_transform_applied'] = True
        else: #print(f"DEBUG: NO se revierte Log {etf_symbol}.");
             results_prophet[etf_symbol]['log_transform_applied'] = False
        print("Guardando forecast (escala original)...")
        forecast_final_scale['pred_int_width']=forecast_final_scale['yhat_upper'] - forecast_final_scale['yhat_lower']
        cols_save=['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'pred_int_width']; cols_pres=[c for c in cols_save if c in forecast_final_scale.columns]
        results_prophet[etf_symbol]['forecast'] = forecast_final_scale[cols_pres].copy()
        #if 'forecast' in results_prophet[etf_symbol]: print(f"DEBUG: Tail guardado:\n{results_prophet[etf_symbol]['forecast'].tail()}")
        #else: print(f"DEBUG: ERROR - 'forecast' no guardado.")
        print(f"Forecast final guardado. Última fecha: {forecast_final_scale['ds'].max().strftime('%Y-%m-%d')}.")

    except Exception as e_pred:
        # Bloque except CON CORRECCIÓN de indentación
        print(f"ERROR generando forecast final {etf_symbol}: {e_pred}")
        results_prophet[etf_symbol]['error_forecast'] = str(e_pred)
        traceback.print_exc() # Imprimir traceback completo del error
        # Limpieza robusta - separar los 'del'
        if 'model_final' in locals():
            del model_final
        if 'df_etf' in locals():
            del df_etf
        if 'df_prophet_train' in locals():
            del df_prophet_train
        if 'future_df_final' in locals():
            del future_df_final
        if 'forecast_log_scale' in locals():
            del forecast_log_scale
        if 'forecast_final_scale' in locals():
            del forecast_final_scale
        gc.collect()
        # El 'continue' también debe estar al mismo nivel
        continue # Saltar al siguiente ETF

    # --- 4.6 Visualización Inicial (Opcional) ---
    # ...

    # --- 4.7 Limpieza --- # <<< SECCIÓN CORREGIDA >>>
    print(f"Limpiando memoria {etf_symbol}...");
    # Eliminar variables grandes una por una si existen
    if 'model_final' in locals():
        del model_final
    if 'df_etf' in locals():
        del df_etf
    if 'df_prophet_train' in locals():
        del df_prophet_train
    if 'future_df_final' in locals():
        del future_df_final
    if 'forecast_log_scale' in locals():
        del forecast_log_scale
    if 'forecast_final_scale' in locals():
        del forecast_final_scale
    if 'best_run' in locals(): # Limpiar variable de tuning también
        del best_run
    if 'tuning_results' in locals(): # Limpiar resultados de tuning
        del tuning_results
    gc.collect(); # Forzar recolección de basura
    print("Limpieza completada.")

    print(f"\n{'='*15} Procesamiento {etf_symbol} completado {'='*15}"); print("-" * 70)

# --- 5. RESUMEN ANÁLISIS ---
print("\n--- Resumen Resultados Análisis ---")
for etf, data in results_prophet.items(): # ... (código resumen sin cambios) ...
    print(f"\n--- ETF: {etf} ---"); print(f"  Params Finales: {data.get('params', 'N/A')}")
    if 'error' in data: print(f"  ERROR: {data['error']}"); continue
    print("\n  Resultado CV/Tuning:"); cv_summary = data.get('cv_metrics_summary'); print(f"    {cv_summary}")
    print("\n  Predicciones Finales (Últimas 5):")
    if 'forecast' in data and not data['forecast'].empty: print(data['forecast'][['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'pred_int_width']].tail().round(4).to_string(index=False))
    elif 'error_forecast' in data: print(f"    ERROR Forecast: {data['error_forecast']}")
    else: print("    No forecast.")
    print("-" * 40)
print("\n--- Fin Resumen Análisis ---")

# --- 6. COMPARACIÓN CON DATOS REALES ---
print("\n\n###########################################################")
print(f"      COMPARACIÓN DATOS REALES DESDE '{ACTUAL_DATA_FILE}'")
print("###########################################################")
try: # ... (carga y preparación datos reales sin cambios) ...
    df_actual_new = pd.read_csv(ACTUAL_DATA_FILE); print(f"Datos reales cargados '{ACTUAL_DATA_FILE}'.")
    DATE_COL_ACTUAL = 'price_date'; VALUE_COL_ACTUAL = 'adj_close'; SYMBOL_COL_ACTUAL = 'fund_symbol'; required_actual_cols = [DATE_COL_ACTUAL, VALUE_COL_ACTUAL, SYMBOL_COL_ACTUAL]
    if not all(col in df_actual_new.columns for col in required_actual_cols): raise ValueError(f"'{ACTUAL_DATA_FILE}' falta cols: {required_actual_cols}")
    df_actual_new[DATE_COL_ACTUAL] = pd.to_datetime(df_actual_new[DATE_COL_ACTUAL]); df_actual_new = df_actual_new.rename(columns={DATE_COL_ACTUAL: 'ds', VALUE_COL_ACTUAL: 'y_actual', SYMBOL_COL_ACTUAL: 'fund_symbol'})
    df_actual_new = df_actual_new.sort_values(by=['fund_symbol', 'ds']); df_actual_new.dropna(subset=['ds', 'y_actual'], inplace=True); print("Datos reales preparados."); comparison_possible = True
except FileNotFoundError: print(f"ERROR: '{ACTUAL_DATA_FILE}' no encontrado."); comparison_possible = False
except Exception as e: print(f"Error cargando/prep '{ACTUAL_DATA_FILE}': {e}"); comparison_possible = False

if comparison_possible:
    print("\n--- Generando Gráficos Comparación (MANUAL + Y-LIM POR PERCENTILES SIN YHAT_UPPER) ---")
    for etf_symbol, results in results_prophet.items():
        print(f"\n--- Comparando para ETF: {etf_symbol} ---")
        if 'forecast' in results and not results['forecast'].empty and 'last_real_date' in results:
            forecast_data_plot = results['forecast'].copy(); last_train_date_plot = results['last_real_date']
            actual_data_etf_plot = df_actual_new[(df_actual_new['fund_symbol'] == etf_symbol) & (df_actual_new['ds'] > last_train_date_plot)].copy()
            if actual_data_etf_plot.empty: print(f"No hay datos reales futuros."); continue
            print(f"Encontrados {len(actual_data_etf_plot)} puntos reales futuros.")
            try: # --- Creación Gráfico Manual ---
                historical_data_orig = df_analysis.loc[df_analysis['fund_symbol'] == etf_symbol, ['price_date', 'adj_close']].rename(columns={'price_date': 'ds', 'adj_close': 'y_hist_orig'}).dropna()
                comparison_df_plot = pd.merge(forecast_data_plot[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], actual_data_etf_plot[['ds', 'y_actual']], on='ds', how='left')
                comparison_df_plot = pd.merge(comparison_df_plot, historical_data_orig[['ds', 'y_hist_orig']], on='ds', how='left')
                fig_comp, ax_comp = plt.subplots(figsize=(14, 7))
                hist_plot_data = comparison_df_plot.dropna(subset=['y_hist_orig'])
                if not hist_plot_data.empty: ax_comp.plot(hist_plot_data['ds'].dt.to_pydatetime(), hist_plot_data['y_hist_orig'], 'k.', label='Observed data points')
                ax_comp.plot(comparison_df_plot['ds'].dt.to_pydatetime(), comparison_df_plot['yhat'], ls='-', c='#0072B2', label='Forecast')
                ax_comp.fill_between(comparison_df_plot['ds'].dt.to_pydatetime(), comparison_df_plot['yhat_lower'], comparison_df_plot['yhat_upper'], color='#0072B2', alpha=0.2, label='Uncertainty interval')
                actual_plot_data = comparison_df_plot.dropna(subset=['y_actual'])
                if not actual_plot_data.empty: ax_comp.plot(actual_plot_data['ds'].dt.to_pydatetime(), actual_plot_data['y_actual'], 'r.', markersize=5, label='Datos Reales (Futuros)')
                else: print(f"Warn: No 'y_actual' válidos.")
                ax_comp.axvline(last_train_date_plot, color='grey', linestyle=':', lw=2, label='Fin Entrenamiento')
                ax_comp.grid(True, which='major', c='gray', ls='-', lw=1, alpha=0.2); ax_comp.set_xlabel('Fecha'); ax_comp.set_ylabel('Valor (adj_close)'); ax_comp.set_title(f'Comparación Predicción vs Realidad para {etf_symbol}')
                days_before = 60; plot_start_date = last_train_date_plot - pd.Timedelta(days=days_before); plot_end_date = comparison_df_plot['ds'].max(); ax_comp.set_xlim([plot_start_date, plot_end_date + pd.Timedelta(days=10)])
                # --- Ajuste Eje Y (SIN yhat_upper) ---
                print("DEBUG: Calculando límites Y basados en hist, actual, yhat, yhat_lower...")
                visible_data_for_lims = comparison_df_plot[(comparison_df_plot['ds'] >= plot_start_date) & (comparison_df_plot['ds'] <= plot_end_date)].copy()
                all_relevant_y_values = pd.concat([visible_data_for_lims['y_hist_orig'], visible_data_for_lims['y_actual'], visible_data_for_lims['yhat'], visible_data_for_lims['yhat_lower'] ]).dropna()
                print(f"DEBUG: Descripción all_relevant_y_values (len={len(all_relevant_y_values)}), SIN yhat_upper:")
                if not all_relevant_y_values.empty:
                    if np.isinf(all_relevant_y_values).any(): all_relevant_y_values = all_relevant_y_values[~np.isinf(all_relevant_y_values)]
                    print(all_relevant_y_values.describe())
                    lower_perc = 0.5; upper_perc = 99.5
                    min_val_p = np.percentile(all_relevant_y_values, lower_perc); max_val_p = np.percentile(all_relevant_y_values, upper_perc)
                    print(f"DEBUG: Percentil {lower_perc}%: {min_val_p:.2f}, {upper_perc}%: {max_val_p:.2f}")
                    y_range_p = max_val_p - min_val_p; y_margin = y_range_p * 0.05
                    if y_margin <= 0: y_margin = max(1, abs(max_val_p * 0.1))
                    final_low = min_val_p - y_margin; final_high = max_val_p + y_margin
                    if final_low < 0 and all_relevant_y_values.min() >= 0: final_low = -y_margin
                    print(f"DEBUG: Límites Y FINALES calculados (sin yhat_upper): [{final_low:.2f}, {final_high:.2f}]")
                    ax_comp.set_ylim([final_low, final_high])
                else: print("DEBUG: No datos válidos para límites Y.")
                # --- Fin Ajuste Eje Y ---
                ax_comp.legend(); plt.tight_layout(); plt.show()
                print(f"Gráfico comparación generado {etf_symbol}.")
            except Exception as e_plot: print(f"Error gráfico {etf_symbol}: {e_plot}"); traceback.print_exc()
        else: # ... (manejo casos sin forecast/fecha) ...
             if 'last_real_date' not in results: print(f"Saltando {etf_symbol}: Falta 'last_real_date'.")
             else: print(f"Saltando {etf_symbol}: No 'forecast'.")
        # Limpieza
        if 'forecast_data_plot' in locals(): del forecast_data_plot;
        if 'actual_data_etf_plot' in locals(): del actual_data_etf_plot;
        if 'comparison_df_plot' in locals(): del comparison_df_plot;
        if 'historical_data_orig' in locals(): del historical_data_orig; gc.collect()
    print("\n--- Fin Comparación ---")
else: print("\nNo se generaron gráficos comparación.")


# --- 7. GUARDAR RESULTADOS PARA EL FRONT-END ---
print("\n\n###########################################################")
print(f"      GUARDANDO RESULTADOS PARA FRONT-END")
print("###########################################################")
results_to_save = {}
if 'results_prophet' in locals() and isinstance(results_prophet, dict):
    print(f"Procesando {len(results_prophet)} ETFs para guardar...")
    for etf, data in results_prophet.items():
        if ('forecast' in data and isinstance(data['forecast'], pd.DataFrame) and
            not data['forecast'].empty and 'ds' in data['forecast'].columns and
            'yhat' in data['forecast'].columns):
            try:
                forecast_df = data['forecast'][['ds', 'yhat']].copy()
                forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])
                forecast_df.dropna(subset=['yhat'], inplace=True)
                if not forecast_df.empty:
                    results_to_save[etf] = {'forecast': forecast_df}
                    print(f"  - Forecast para {etf} preparado ({len(forecast_df)} filas).")
                else: print(f"  - WARN: Forecast {etf} vacío post-limpieza. No guardado.")
            except Exception as e_prep_save: print(f"  - ERROR prep forecast {etf}: {e_prep_save}.")
        else: print(f"  - WARN: No se guardará {etf} (sin forecast válido).")

    if results_to_save:
        output_filename = 'prophet_frontend_data.pkl'
        try:
            with open(output_filename, 'wb') as f: pickle.dump(results_to_save, f)
            print(f"\nResultados ({len(results_to_save)} ETFs) guardados en: '{output_filename}'")
            print(f"  ETFs guardados: {list(results_to_save.keys())}")
        except Exception as e_save: print(f"\nERROR guardando resultados: {e_save}"); traceback.print_exc()
    else: print("\nNo se encontraron resultados válidos para guardar.")
else: print("\nVariable 'results_prophet' no encontrada. No se guardó nada.")

print("\n--- Fin Guardado para Front-End ---")
print("\n--- Script de Análisis Finalizado ---")

Usando 'df_top20'.
'price_date' entrenamiento a datetime.

Identificando 5 ETFs con más datos...
Se procesarán 5 ETFs (más datos):
  1. SPY (7263 puntos)
  2. QLD (3889 puntos)
  3. ROM (3735 puntos)
  4. RXL (3735 puntos)
  5. USD (3735 puntos)
--------------------------------------------------

=============== Procesando ETF: SPY ===============
Log Transform aplicada SPY.
Entren. listo. Última fecha: 2021-11-30. Puntos: 7263

--- Iniciando Tuning SPY ---


Streaming output truncated to the last 5000 lines.
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=577', 'data', 'file=/tmp/tmpowhd_98b/pn_imn25.json', 'init=/tmp/tmpowhd_98b/0lhefciu.json', 'output', 'file=/tmp/tmpowhd_98b/prophet_modelx1w2fopr/prophet_model-20250424165909.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:59:09 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:59:10 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpowhd_98b/4f8jrgd0.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpowhd_98b/ljpjgvag.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=37634', 'data',

In [ ]:
# -*- coding: utf-8 -*-
"""
calculadora_etf_app.py

Calculadora de Interés Compuesto basada en Predicciones Prophet de ETFs.
Aplicación Front-End con Streamlit.
"""

import streamlit as st
import pandas as pd
import numpy as np
import pickle
from datetime import timedelta
import plotly.graph_objects as go
import os # Para listar archivos (depuración)

# --- Configuración de la Página ---
st.set_page_config(
    page_title="Calculadora Compuesta ETF (Predicción)",
    page_icon="📈",
    layout="wide"
)

# --- Carga de Datos Pre-Calculados ---
@st.cache_data # Cache para eficiencia
def load_results(filepath='prophet_frontend_data.pkl'):
    """Carga y valida los resultados guardados."""
    st.write(f"Intentando cargar datos desde: {filepath}") # Mensaje de carga
    if not os.path.exists(filepath):
        st.error(f"ERROR CRÍTICO: Archivo '{filepath}' no encontrado.")
        st.info("Asegúrate de haber ejecutado el script de análisis y que el archivo .pkl está en el mismo directorio.")
        try: st.info(f"Archivos en directorio actual: {os.listdir('.')}")
        except Exception: pass
        return None

    try:
        with open(filepath, 'rb') as f:
            results = pickle.load(f)
        st.write("Archivo .pkl cargado.")

        if not isinstance(results, dict) or not results:
            st.error("Error: Archivo .pkl vacío o formato incorrecto (no es diccionario).")
            return None

        valid_etfs = {}
        st.write(f"Validando {len(results)} ETFs del archivo...")
        for etf, data in results.items():
            valid = False
            if (isinstance(data, dict) and 'forecast' in data and
                isinstance(data['forecast'], pd.DataFrame) and not data['forecast'].empty and
                'ds' in data['forecast'].columns and 'yhat' in data['forecast'].columns):
                try:
                    data['forecast']['ds'] = pd.to_datetime(data['forecast']['ds'])
                    data['forecast'].dropna(subset=['yhat'], inplace=True) # Asegurar no NaNs en yhat
                    if not data['forecast'].empty:
                        valid_etfs[etf] = data
                        valid = True
                        # st.write(f"  - {etf}: OK") # Descomentar para más detalle
                    else: st.warning(f"  - {etf}: Forecast vacío post-limpieza.")
                except Exception as e_dt: st.warning(f"  - {etf}: Error formato/datos: {e_dt}.")
            else: st.warning(f"  - {etf}: Datos inválidos/incompletos.")
            #if not valid: st.warning(f"  - {etf}: NO pasó validación.")

        if not valid_etfs:
            st.error("No se encontraron ETFs con datos válidos en el archivo.")
            return None

        st.success(f"Carga y validación completada. {len(valid_etfs)} ETFs disponibles.")
        return valid_etfs

    except Exception as e:
        st.error(f"Error inesperado al cargar '{filepath}': {e}")
        import traceback
        st.text(traceback.format_exc())
        return None

# --- Ejecución Principal de la App ---
prophet_results = load_results() # Cargar datos al inicio

st.title("📈 Calculadora de Interés Compuesto con Predicciones Prophet")

if prophet_results is None:
    st.subheader("Error al cargar datos")
    st.warning("No se pudieron cargar los datos de predicción. Verifica que el archivo "
               "'prophet_frontend_data.pkl' existe y es válido.")
    st.stop() # Detener si no hay datos

# --- UI Principal (si los datos cargaron bien) ---
st.markdown("Simula el crecimiento de una inversión usando los **retornos predichos** por Prophet.")

available_etfs = sorted(list(prophet_results.keys()))
if not available_etfs:
    st.error("No hay ETFs disponibles en los datos cargados.")
    st.stop()

# --- Inputs Usuario ---
col1, col2, col3 = st.columns(3)
with col1:
    selected_etf = st.selectbox("1. Selecciona ETF:", available_etfs, index=0)
    initial_investment = st.number_input("2. Inversión Inicial (€/$):", min_value=0.0, value=1000.0, step=100.0, format="%.2f")
with col2:
    monthly_contribution = st.number_input("3. Aportación Mensual (€/$):", min_value=0.0, value=100.0, step=50.0, format="%.2f")
    # Lógica para max_years_input (igual que antes)
    try:
        max_forecast_date_overall = max(data['forecast']['ds'].max() for data in prophet_results.values())
        min_forecast_date_overall = min(data['forecast']['ds'].min() for data in prophet_results.values())
        max_years_possible = int((max_forecast_date_overall - min_forecast_date_overall).days / 365)
        max_years_input = min(max_years_possible, 15) # Aumentado límite práctico
        if max_years_input < 1: max_years_input = 1
    except: # Fallback por si algo falla
        max_years_input = 10
    investment_years = st.number_input(f"4. Duración (Años - Máx. {max_years_input}):", min_value=1, max_value=max_years_input, value=min(3, max_years_input), step=1)

with col3:
    st.markdown("<br/><br/>", unsafe_allow_html=True) # Espacio
    calculate_button = st.button("🚀 Calcular Proyección", use_container_width=True)

st.divider()

# --- Cálculos y Resultados (si se presiona el botón) ---
if calculate_button and selected_etf:
    st.header(f"Proyección para {selected_etf}")
    try:
        forecast_data = prophet_results[selected_etf]['forecast'].copy().sort_values('ds').set_index('ds')
        start_date = forecast_data.index.min()
        end_date_sim = start_date + pd.DateOffset(years=investment_years)
        max_forecast_date = forecast_data.index.max()

        if end_date_sim > max_forecast_date:
            st.warning(f"Duración excede forecast. Limitando a {max_forecast_date.strftime('%Y-%m-%d')}.")
            end_date_sim = max_forecast_date
            actual_sim_years = (end_date_sim - start_date).days / 365.25
            st.info(f"Simulación por aprox. {actual_sim_years:.1f} años.")

        sim_forecast = forecast_data[(forecast_data.index >= start_date) & (forecast_data.index <= end_date_sim)].copy()

        if sim_forecast.empty: st.error(f"No hay predicciones para {selected_etf} en el período."); st.stop()

        # --- Lógica Cálculo ---
        # Resample a inicio de mes para calcular retorno del mes anterior
        monthly_prices_start = sim_forecast['yhat'].resample('MS').first()
        monthly_returns = monthly_prices_start.pct_change() # Retorno vs inicio mes anterior
        # El primer retorno es NaN, lo tratamos como 0 o podríamos omitir ese primer mes de cálculo de retorno
        monthly_returns = monthly_returns.fillna(0)

        current_balance = initial_investment
        investment_history = pd.DataFrame(columns=['Balance', 'Contribuciones', 'Crecimiento_Mes'])
        # Registrar estado inicial justo antes del primer mes de simulación
        record_date_initial = start_date.to_period('M').start_time - timedelta(days=1) # Fin del mes anterior al inicio
        investment_history.loc[record_date_initial] = [initial_investment, initial_investment, 0.0]

        # Iterar por los inicios de mes para aplicar retorno y añadir contribución
        for month_start_date, monthly_return in monthly_returns.items():
            # No calcular retorno para el primer mes (ya es NaN o 0)
            if month_start_date == monthly_returns.index.min():
                 balance_before_contrib = current_balance
                 growth_this_month = 0 # Sin crecimiento el primer mes
            else:
                 # Calcular crecimiento basado en el balance *antes* de la contribución de este mes
                 balance_before_contrib = current_balance
                 growth_this_month = balance_before_contrib * monthly_return
                 current_balance += growth_this_month

            # Añadir contribución mensual (excepto en el estado inicial)
            if month_start_date > record_date_initial:
                current_balance += monthly_contribution
                total_contrib_so_far = investment_history['Contribuciones'].iloc[-1] + monthly_contribution
            else: # Estado inicial
                 total_contrib_so_far = initial_investment


            # Guardar balance y contribuciones al final de este "período" (inicio de mes)
            investment_history.loc[month_start_date] = [current_balance, total_contrib_so_far, growth_this_month]

        # --- Resultados ---
        final_balance = investment_history['Balance'].iloc[-1]
        total_contributions = investment_history['Contribuciones'].iloc[-1]
        total_growth = final_balance - total_contributions

        st.subheader("Resultados Simulación")
        col_res1, col_res2, col_res3 = st.columns(3)
        with col_res1: st.metric("Valor Final Proyectado", f"{final_balance:,.2f} €/$")
        with col_res2: st.metric("Aportaciones Totales", f"{total_contributions:,.2f} €/$")
        with col_res3: st.metric("Crecimiento Estimado (Predicción)", f"{total_growth:,.2f} €/$", delta=f"{total_growth/total_contributions*100 if total_contributions>0 else 0:.1f}%")

        # --- Gráfico ---
        st.subheader("Gráfico Crecimiento Proyectado")
        investment_history.index = pd.to_datetime(investment_history.index)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=investment_history.index, y=investment_history['Balance'], mode='lines', name='Valor Inversión', line=dict(color='green', width=2), hovertemplate='<b>Fecha</b>: %{x|%Y-%m-%d}<br><b>Balance</b>: %{y:,.2f} €/$<extra></extra>'))
        # Opcional: Añadir línea de contribuciones
        # fig.add_trace(go.Scatter(x=investment_history.index, y=investment_history['Contribuciones'], mode='lines', name='Aportaciones', line=dict(color='gray', dash='dash'), hovertemplate='<b>Fecha</b>: %{x|%Y-%m-%d}<br><b>Aportado</b>: %{y:,.2f} €/$<extra></extra>'))
        fig.update_layout(title=f'Crecimiento Inversión en {selected_etf} (Basado en Predicción)', xaxis_title='Fecha', yaxis_title='Valor (€/$)', xaxis_tickformat='%Y-%m-%d', hovermode='x unified', legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
        st.plotly_chart(fig, use_container_width=True)

    except Exception as e_calc:
        st.error(f"Error durante el cálculo o graficación: {e_calc}")
        st.text(traceback.format_exc())

# --- Footer ---
st.sidebar.markdown("---")
st.sidebar.info("Esta calculadora usa predicciones de Prophet. Los resultados son ilustrativos y no garantizan rendimientos futuros.")

## V2

In [ ]:
# --- Instalación de Librerías ---
print("Instalando librerías necesarias (Prophet, Streamlit, Pyngrok)...")
!pip install prophet cmdstanpy streamlit pyngrok -q
print("Instalación completada.")

# --- Imports para la Parte de Análisis ---
import pandas as pd
import numpy as np
# Quitamos matplotlib de aquí ya que el plot final se hará en Streamlit
# import matplotlib.pyplot as plt
import warnings
import gc
import itertools
import traceback
import pickle
import os

# Configuración de advertencias
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

# Imports de Prophet (verificar después de instalar)
try:
    from prophet import Prophet
    from prophet.diagnostics import cross_validation, performance_metrics
    print("Importaciones de Prophet correctas.")
except ImportError:
    print("*"*30); print("ERROR: Faltan componentes de Prophet. Intenta reiniciar el entorno y reinstalar."); print("*"*30); exit()

In [ ]:
# --- 1. CONFIGURACIÓN GENERAL ---
TOP_N_ETFS = 5
# !!! ASEGÚRATE DE QUE ESTA VARIABLE (DataFrame) EXISTA EN TU ENTORNO !!!
# Por ejemplo, si cargas desde un CSV:
# df_top20 = pd.read_csv('tu_archivo_entrenamiento.csv')
DATASET_VARIABLE_NAME = 'df_top20' # Variable que contiene tus datos de entrenamiento
FUTURE_END_DATE = '2025-03-10'
ACTUAL_DATA_FILE = 'ETF_cohortes.csv' # Archivo con datos completos/actuales (para regresores futuros si se usan)
FRONTEND_DATA_FILE = 'prophet_frontend_data.pkl' # Archivo donde se guardan resultados para Streamlit

# --- 1.1 CONFIGURACIÓN DE MEJORAS ---
USE_LOG_TRANSFORM = True
REGRESSOR_COLUMNS = [] # Añade nombres de columnas si usas regresores
PERFORM_HYPERPARAMETER_TUNING = False # Poner a True para ejecutar tuning (lento)
PARAM_GRID = { 'changepoint_prior_scale': [0.01, 0.1, 0.5], 'seasonality_prior_scale': [1.0, 10.0, 20.0], 'holidays_prior_scale': [1.0, 10.0, 20.0] }
PROPHET_BASE_PARAMS = { 'seasonality_mode': 'multiplicative', 'daily_seasonality': False, 'weekly_seasonality': True, 'yearly_seasonality': True, 'interval_width': 0.80 }
CUSTOM_HOLIDAYS = None

# --- 1.2 CONFIGURACIÓN CV ---
EJECUTAR_CROSS_VALIDATION = False # Poner a True SOLO si quieres métricas CV (o para tuning)
CV_INITIAL = '730 days'; CV_PERIOD = '90 days'; CV_HORIZON = '180 days'; CV_METRIC_TO_OPTIMIZE = 'rmse'

print("Configuración cargada.")

In [ ]:
# --- INICIO SCRIPT ANÁLISIS PROPHET ---

# --- 2. CARGA Y PREPARACIÓN DATAFRAME ENTRENAMIENTO ---
print("\n--- Iniciando Análisis Prophet ---")
if DATASET_VARIABLE_NAME not in globals(): raise NameError(f"'{DATASET_VARIABLE_NAME}' no encontrado.")
else: df_analysis = globals()[DATASET_VARIABLE_NAME].copy(); print(f"Usando '{DATASET_VARIABLE_NAME}'.")
required_cols_train = ['fund_symbol', 'price_date', 'adj_close'] + REGRESSOR_COLUMNS
missing_cols = [col for col in required_cols_train if col not in df_analysis.columns]
if missing_cols: raise ValueError(f"Faltan cols en {DATASET_VARIABLE_NAME}: {missing_cols}")
try: df_analysis['price_date'] = pd.to_datetime(df_analysis['price_date']); print("'price_date' entrenamiento a datetime.")
except Exception as e: print(f"Error convirtiendo fecha entreno: {e}"); exit()
df_analysis.dropna(subset=['adj_close'], inplace=True)
if REGRESSOR_COLUMNS: # ... (Lógica relleno NaNs regresores entrenamiento) ...
    for col in REGRESSOR_COLUMNS:
        df_analysis[col] = df_analysis.groupby('fund_symbol')[col].ffill().bfill()
        if df_analysis[col].isnull().any(): print(f"WARN: NaNs regresor entreno '{col}'.")

# --- 3. SELECCIONAR TOP N ETFs POR NÚMERO DE DATOS ---
print(f"\nIdentificando {TOP_N_ETFS} ETFs con más datos...")
etf_data_counts = df_analysis.groupby('fund_symbol').size().reset_index(name='data_count')
etf_data_counts_sorted = etf_data_counts.sort_values(by='data_count', ascending=False)
top_N_etfs_list = etf_data_counts_sorted['fund_symbol'].head(TOP_N_ETFS).tolist()
if not top_N_etfs_list: raise ValueError("No se pudo contar datos.");
if len(top_N_etfs_list) < TOP_N_ETFS: print(f"Warn: Solo {len(top_N_etfs_list)} ETFs."); TOP_N_ETFS = len(top_N_etfs_list)
print(f"Se procesarán {TOP_N_ETFS} ETFs (más datos):")
selected_counts_info = etf_data_counts_sorted.head(TOP_N_ETFS)
for i, (index, row) in enumerate(selected_counts_info.iterrows()): print(f"  {i+1}. {row['fund_symbol']} ({row['data_count']} puntos)")
print("-" * 50)

# --- 4. BUCLE PRINCIPAL PARA PROCESAR CADA ETF ---
results_prophet = {}
for etf_symbol in top_N_etfs_list:
    print(f"\n{'='*15} Procesando ETF: {etf_symbol} {'='*15}")
    results_prophet[etf_symbol] = {'params': {}}; is_log_transformed_etf = False
    # --- 4.1 Prep Datos Entrenamiento ---
    df_etf = df_analysis[df_analysis['fund_symbol'] == etf_symbol].copy().sort_values(by='price_date').reset_index(drop=True)
    cols_to_keep = ['price_date', 'adj_close'] + REGRESSOR_COLUMNS
    df_prophet_train = df_etf[cols_to_keep].rename(columns={'price_date': 'ds', 'adj_close': 'y'}).dropna(subset=['ds', 'y'])
    if USE_LOG_TRANSFORM:
        if (df_prophet_train['y'] <= 0).any(): print(f"WARN: Valores <= 0 {etf_symbol}. Saltando Log.")
        else: df_prophet_train['y'] = np.log(df_prophet_train['y']); is_log_transformed_etf = True; print(f"Log Transform aplicada {etf_symbol}.")
    min_data_points_total = 30; can_run_cv = True
    try: # ... (lógica check datos CV) ...
        min_days_cv = pd.Timedelta(CV_INITIAL).days + pd.Timedelta(CV_HORIZON).days; data_days = (df_prophet_train['ds'].max() - df_prophet_train['ds'].min()).days
        if len(df_prophet_train) < min_data_points_total: raise ValueError(f"Insuficientes ({len(df_prophet_train)})")
        if data_days < min_days_cv: print(f"WARN: Rango ({data_days} días) corto para CV."); can_run_cv = False
    except Exception as e_check: print(f"ERROR check datos {etf_symbol}: {e_check}"); results_prophet[etf_symbol]['error'] = f"Err check: {e_check}"; del df_etf, df_prophet_train; gc.collect(); continue
    last_train_date = df_prophet_train['ds'].max(); results_prophet[etf_symbol]['last_real_date'] = last_train_date
    print(f"Entren. listo. Última fecha: {last_train_date.strftime('%Y-%m-%d')}. Puntos: {len(df_prophet_train)}")

    # --- 4.2 Tuning ---
    best_params_for_etf = PROPHET_BASE_PARAMS.copy(); best_run = None
    if PERFORM_HYPERPARAMETER_TUNING and EJECUTAR_CROSS_VALIDATION and can_run_cv:
        # ... (lógica completa del tuning, igual que antes) ...
        print(f"\n--- Iniciando Tuning {etf_symbol} ---")
        param_combinations = [dict(zip(PARAM_GRID.keys(), v)) for v in itertools.product(*PARAM_GRID.values())]; tuning_results = []
        for params_to_try in param_combinations:
            current_trial_params = {**PROPHET_BASE_PARAMS, **params_to_try}
            try:
                m_tune = Prophet(**current_trial_params, holidays=CUSTOM_HOLIDAYS)
                if REGRESSOR_COLUMNS: [m_tune.add_regressor(r) for r in REGRESSOR_COLUMNS]
                m_tune.fit(df_prophet_train)
                df_cv_tune = cross_validation(m_tune, initial=CV_INITIAL, period=CV_PERIOD, horizon=CV_HORIZON, parallel="processes", disable_tqdm=True)
                df_p_tune = performance_metrics(df_cv_tune); metric_value = df_p_tune[CV_METRIC_TO_OPTIMIZE].mean()
                tuning_results.append({'params': params_to_try, 'metric': metric_value})
                del m_tune, df_cv_tune, df_p_tune; gc.collect()
            except Exception as e_tune: print(f"  ERROR CV {params_to_try}: {e_tune}"); tuning_results.append({'params': params_to_try, 'metric': float('inf')})
        if tuning_results:
            best_run = min(tuning_results, key=lambda x: x['metric'])
            if best_run['metric'] != float('inf'): best_params_for_etf.update(best_run['params']); print(f"\nMejores params {etf_symbol}: {best_run['params']} ({CV_METRIC_TO_OPTIMIZE}={best_run['metric']:.4f})")
            else: print("\nTuning falló. Usando base.")
        else: print("\nNo resultados tuning. Usando base.")
    else: # Mensajes de omisión
        if not (PERFORM_HYPERPARAMETER_TUNING and EJECUTAR_CROSS_VALIDATION): print("\nTuning OMITIDO (desactivado).")
        elif not can_run_cv: print("\nTuning OMITIDO (datos insuficientes).")
    results_prophet[etf_symbol]['params'] = best_params_for_etf.copy(); print(f"Params finales {etf_symbol}: {best_params_for_etf}")

    # --- 4.3 Modelo Final ---
    print(f"\nEntrenando modelo FINAL {etf_symbol}..."); model_final = Prophet(**best_params_for_etf, holidays=CUSTOM_HOLIDAYS)
    if REGRESSOR_COLUMNS: print(f"Añadiendo regresores: {REGRESSOR_COLUMNS}"); [model_final.add_regressor(r) for r in REGRESSOR_COLUMNS]
    try: model_final.fit(df_prophet_train); print("Entrenamiento final completado.")
    except Exception as e_fit_final: # ... (manejo de error igual que antes, con indentación corregida) ...
        print(f"ERROR entrenando FINAL: {e_fit_final}"); results_prophet[etf_symbol]['error'] = f"Err entreno final: {e_fit_final}";
        if 'df_etf' in locals(): del df_etf
        if 'df_prophet_train' in locals(): del df_prophet_train
        gc.collect();
        if 'model_final' in locals(): del model_final
        continue

    # --- 4.4 Resumen CV/Tuning ---
    # ... (lógica igual que antes) ...
    if best_run and best_run['metric'] != float('inf') and PERFORM_HYPERPARAMETER_TUNING: results_prophet[etf_symbol]['cv_metrics_summary'] = f"Mejor {CV_METRIC_TO_OPTIMIZE} Tuning: {best_run['metric']:.4f}"
    elif not can_run_cv: results_prophet[etf_symbol]['cv_metrics_summary'] = "CV Omitida (datos insuf.)"
    elif not EJECUTAR_CROSS_VALIDATION: results_prophet[etf_symbol]['cv_metrics_summary'] = "CV Omitida (desactivada)"
    else: results_prophet[etf_symbol]['cv_metrics_summary'] = "CV no ejecutada (sin tuning)"

    # --- 4.5 Forecast Final ---
    print(f"\nGenerando Forecast {etf_symbol}..."); forecast_log_scale = None; future_df_final = None; forecast_final_scale = None
    try: # ... (lógica forecast igual que antes, con manejo robusto log y regresores) ...
        target_end_date = pd.to_datetime(FUTURE_END_DATE); freq = 'B'
        future_dates = pd.date_range(start=last_train_date + pd.Timedelta(days=1), end=target_end_date, freq=freq)
        future_df_base = pd.DataFrame({'ds': future_dates}); future_df_final = future_df_base.copy()
        if REGRESSOR_COLUMNS: # ... (lógica añadir regresores futuros) ...
             print("Añadiendo regresores futuros...");
             try: # ... (igual que antes) ...
                 df_actual_full=pd.read_csv(ACTUAL_DATA_FILE); df_actual_full['ds']=pd.to_datetime(df_actual_full['price_date'])
                 df_reg_fut=df_actual_full.loc[df_actual_full['fund_symbol'] == etf_symbol, ['ds'] + REGRESSOR_COLUMNS].copy()
                 future_df_final=pd.merge(future_df_final, df_reg_fut, on='ds', how='left')
                 for col in REGRESSOR_COLUMNS:
                     future_df_final[col]=future_df_final[col].ffill().bfill()
                     if future_df_final[col].isnull().any(): last_known = df_prophet_train[col].iloc[-1]; future_df_final[col].fillna(last_known, inplace=True); print(f"WARN: NaNs reg futuro '{col}' rellenados.")
                 del df_actual_full, df_reg_fut
             except Exception as e_reg_fut: print(f"ERROR obteniendo regresores futuros: {e_reg_fut}"); raise
        print("Realizando predicción..."); future_df_final = future_df_final[future_df_final['ds'] <= target_end_date];
        if future_df_final.empty: raise ValueError("Future DF vacío.")
        forecast_log_scale = model_final.predict(future_df_final); print("Predicción completada.")
        forecast_final_scale = forecast_log_scale.copy()
        if is_log_transformed_etf: # ... (lógica reversión log) ...
             cols_to_exp=['yhat', 'yhat_lower', 'yhat_upper','trend', 'trend_lower', 'trend_upper']
             for col in cols_to_exp:
                 if col in forecast_final_scale.columns:
                     try: forecast_final_scale[col]=np.exp(forecast_final_scale[col])
                     except Exception as e_exp: print(f"ERROR np.exp '{col}': {e_exp}")
             results_prophet[etf_symbol]['log_transform_applied'] = True
        else: results_prophet[etf_symbol]['log_transform_applied'] = False
        print("Guardando forecast (escala original)...")
        forecast_final_scale['pred_int_width']=forecast_final_scale['yhat_upper'] - forecast_final_scale['yhat_lower']
        cols_save=['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'pred_int_width']; cols_pres=[c for c in cols_save if c in forecast_final_scale.columns]
        results_prophet[etf_symbol]['forecast'] = forecast_final_scale[cols_pres].copy() # Guardar DF procesado
        print(f"Forecast final guardado. Última fecha: {forecast_final_scale['ds'].max().strftime('%Y-%m-%d')}.")
    except Exception as e_pred: # ... (manejo error forecast con indentación corregida) ...
        print(f"ERROR generando forecast final {etf_symbol}: {e_pred}"); results_prophet[etf_symbol]['error_forecast'] = str(e_pred); traceback.print_exc();
        if 'model_final' in locals(): del model_final
        if 'df_etf' in locals(): del df_etf
        if 'df_prophet_train' in locals(): del df_prophet_train
        if 'future_df_final' in locals(): del future_df_final
        if 'forecast_log_scale' in locals(): del forecast_log_scale
        if 'forecast_final_scale' in locals(): del forecast_final_scale
        gc.collect(); continue

    # --- 4.7 Limpieza ---
    print(f"Limpiando memoria {etf_symbol}..."); # ... (limpieza igual que antes, con indentación corregida) ...
    if 'model_final' in locals(): del model_final
    if 'df_etf' in locals(): del df_etf
    if 'df_prophet_train' in locals(): del df_prophet_train
    if 'future_df_final' in locals(): del future_df_final
    if 'forecast_log_scale' in locals(): del forecast_log_scale
    if 'forecast_final_scale' in locals(): del forecast_final_scale
    if 'best_run' in locals(): del best_run
    if 'tuning_results' in locals(): del tuning_results
    gc.collect(); print("Limpieza completada.")
    print(f"\n{'='*15} Procesamiento {etf_symbol} completado {'='*15}"); print("-" * 70)

# --- 5. RESUMEN ANÁLISIS (Opcional, puedes comentarlo si no lo necesitas) ---
print("\n--- Resumen Resultados Análisis ---")
# ... (código resumen sin cambios) ...
for etf, data in results_prophet.items():
    print(f"\n--- ETF: {etf} ---"); print(f"  Params Finales: {data.get('params', 'N/A')}")
    if 'error' in data: print(f"  ERROR: {data['error']}"); continue
    print("\n  Resultado CV/Tuning:"); cv_summary = data.get('cv_metrics_summary'); print(f"    {cv_summary}")
    print("\n  Predicciones Finales (Últimas 5):")
    if 'forecast' in data and not data['forecast'].empty: print(data['forecast'][['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'pred_int_width']].tail().round(4).to_string(index=False))
    elif 'error_forecast' in data: print(f"    ERROR Forecast: {data['error_forecast']}")
    else: print("    No forecast.")
    print("-" * 40)
print("\n--- Fin Resumen Análisis ---")

# --- 6. COMPARACIÓN CON DATOS REALES (Opcional) ---
# Comenta esta sección si no necesitas los gráficos de matplotlib aquí
# print("\n\n###########################################################")
# print(f"      COMPARACIÓN DATOS REALES DESDE '{ACTUAL_DATA_FILE}'")
# # ... (código completo Sección 6 anterior si quieres los plots) ...
# print("\n--- Fin Comparación ---")

# --- 7. GUARDAR RESULTADOS PARA EL FRONT-END --- # <<< SECCIÓN CLAVE >>>
print("\n\n###########################################################")
print(f"      GUARDANDO RESULTADOS PARA FRONT-END en '{FRONTEND_DATA_FILE}'")
print("###########################################################")
results_to_save = {}
if 'results_prophet' in locals() and isinstance(results_prophet, dict):
    print(f"Procesando {len(results_prophet)} ETFs para guardar...")
    for etf, data in results_prophet.items():
        if ('forecast' in data and isinstance(data['forecast'], pd.DataFrame) and
            not data['forecast'].empty and 'ds' in data['forecast'].columns and
            'yhat' in data['forecast'].columns):
            try:
                forecast_df = data['forecast'][['ds', 'yhat']].copy()
                forecast_df['ds'] = pd.to_datetime(forecast_df['ds'])
                forecast_df.dropna(subset=['yhat'], inplace=True)
                if not forecast_df.empty:
                    results_to_save[etf] = {'forecast': forecast_df}
                    print(f"  - Forecast para {etf} preparado ({len(forecast_df)} filas).")
                else: print(f"  - WARN: Forecast {etf} vacío post-limpieza. No guardado.")
            except Exception as e_prep_save: print(f"  - ERROR prep forecast {etf}: {e_prep_save}.")
        else: print(f"  - WARN: No se guardará {etf} (sin forecast válido).")
    if results_to_save:
        try:
            with open(FRONTEND_DATA_FILE, 'wb') as f: pickle.dump(results_to_save, f)
            print(f"\nResultados ({len(results_to_save)} ETFs) guardados en: '{FRONTEND_DATA_FILE}'")
            print(f"  ETFs guardados: {list(results_to_save.keys())}")
        except Exception as e_save: print(f"\nERROR guardando resultados: {e_save}"); traceback.print_exc()
    else: print("\nNo se encontraron resultados válidos para guardar.")
else: print("\nVariable 'results_prophet' no encontrada. No se guardó nada.")
print("\n--- Fin Guardado para Front-End ---")

print("\n--- Script de Análisis Finalizado ---")

In [ ]:
# --- Definición del Código Streamlit ---
streamlit_app_code = """
# -*- coding: utf-8 -*-
# calculadora_etf_app.py

import streamlit as st
import pandas as pd
import numpy as np
import pickle
from datetime import timedelta
import plotly.graph_objects as go
import os # Para listar archivos (depuración)

# Configuración Página
st.set_page_config(page_title="Calculadora Compuesta ETF", page_icon="📈", layout="wide")

# Carga de Datos (con cache y validación)
@st.cache_data
def load_results(filepath='prophet_frontend_data.pkl'):
    st.write(f"Intentando cargar datos desde: {filepath}")
    if not os.path.exists(filepath):
        st.error(f"ERROR CRÍTICO: Archivo '{filepath}' no encontrado.")
        st.info("Ejecuta el script de análisis para generarlo.")
        try: st.info(f"Archivos directorio actual: {os.listdir('.')}")
        except Exception: pass
        return None
    try:
        with open(filepath, 'rb') as f: results = pickle.load(f)
        st.write("Archivo .pkl cargado.")
        if not isinstance(results, dict) or not results:
            st.error("Error: Archivo .pkl vacío o formato incorrecto."); return None
        valid_etfs = {}
        st.write(f"Validando {len(results)} ETFs del archivo...")
        for etf, data in results.items():
            valid = False
            if (isinstance(data, dict) and 'forecast' in data and
                isinstance(data['forecast'], pd.DataFrame) and not data['forecast'].empty and
                'ds' in data['forecast'].columns and 'yhat' in data['forecast'].columns):
                try:
                    data['forecast']['ds'] = pd.to_datetime(data['forecast']['ds'])
                    data['forecast'].dropna(subset=['yhat'], inplace=True)
                    if not data['forecast'].empty:
                        valid_etfs[etf] = data; valid = True
                except Exception as e_dt: st.warning(f"Error formato/datos {etf}: {e_dt}.")
            if not valid: st.warning(f"Datos inválidos/incompletos {etf}.")
        if not valid_etfs: st.error("No se encontraron ETFs válidos."); return None
        st.success(f"Carga completada. {len(valid_etfs)} ETFs disponibles.")
        return valid_etfs
    except Exception as e: st.error(f"Error inesperado cargando: {e}"); import traceback; st.text(traceback.format_exc()); return None

# Ejecución Principal App
prophet_results = load_results() # Usar nombre de archivo definido en Cfg Análisis

st.title("📈 Calculadora de Interés Compuesto con Predicciones Prophet")

if prophet_results is None:
    st.subheader("Error al cargar datos")
    st.warning("No se pudieron cargar datos de predicción. Verifica el archivo .pkl.")
    st.stop()

st.markdown("Simula el crecimiento de inversión usando **retornos predichos**.")
available_etfs = sorted(list(prophet_results.keys()))
if not available_etfs: st.error("No hay ETFs disponibles."); st.stop()

# Inputs Usuario
col1, col2, col3 = st.columns(3)
with col1:
    selected_etf = st.selectbox("1. Selecciona ETF:", available_etfs, index=0)
    initial_investment = st.number_input("2. Inversión Inicial (€/$):", min_value=0.0, value=1000.0, step=100.0, format="%.2f")
with col2:
    monthly_contribution = st.number_input("3. Aportación Mensual (€/$):", min_value=0.0, value=100.0, step=50.0, format="%.2f")
    try: # Lógica max_years_input
        max_date = max(d['forecast']['ds'].max() for d in prophet_results.values())
        min_date = min(d['forecast']['ds'].min() for d in prophet_results.values())
        max_years_poss = int((max_date - min_date).days / 365); max_yrs_in = min(max_years_poss, 15)
        if max_yrs_in < 1: max_yrs_in = 1
    except: max_yrs_in = 10
    investment_years = st.number_input(f"4. Duración (Años - Máx. {max_yrs_in}):", min_value=1, max_value=max_yrs_in, value=min(3, max_yrs_in), step=1)
with col3: st.markdown("<br/><br/>", unsafe_allow_html=True); calculate_button = st.button("🚀 Calcular Proyección", use_container_width=True)
st.divider()

# Cálculos y Resultados
if calculate_button and selected_etf:
    st.header(f"Proyección para {selected_etf}")
    try:
        forecast_data = prophet_results[selected_etf]['forecast'].copy().sort_values('ds').set_index('ds')
        start_date = forecast_data.index.min(); end_date_sim = start_date + pd.DateOffset(years=investment_years); max_forecast_date = forecast_data.index.max()
        if end_date_sim > max_forecast_date: st.warning(f"Limitando simulación a {max_forecast_date.strftime('%Y-%m-%d')}."); end_date_sim = max_forecast_date;
        sim_forecast = forecast_data[(forecast_data.index >= start_date) & (forecast_data.index <= end_date_sim)].copy()
        if sim_forecast.empty: st.error(f"No hay predicciones para {selected_etf} en período."); st.stop()

        # Lógica Cálculo Compuesto
        monthly_prices_start = sim_forecast['yhat'].resample('MS').first(); monthly_returns = monthly_prices_start.pct_change().fillna(0)
        current_balance = initial_investment; investment_history = pd.DataFrame(columns=['Balance', 'Contribuciones', 'Crecimiento_Mes'])
        record_date_initial = start_date.to_period('M').start_time - timedelta(days=1)
        investment_history.loc[record_date_initial] = [initial_investment, initial_investment, 0.0]
        for month_start_date, monthly_return in monthly_returns.items():
            balance_before_contrib = current_balance; growth_this_month = 0
            if month_start_date != monthly_returns.index.min(): growth_this_month = balance_before_contrib * monthly_return; current_balance += growth_this_month
            if month_start_date > record_date_initial: current_balance += monthly_contribution; total_contrib_so_far = investment_history['Contribuciones'].iloc[-1] + monthly_contribution
            else: total_contrib_so_far = initial_investment
            investment_history.loc[month_start_date] = [current_balance, total_contrib_so_far, growth_this_month]

        # Resultados
        final_balance = investment_history['Balance'].iloc[-1]; total_contributions = investment_history['Contribuciones'].iloc[-1]; total_growth = final_balance - total_contributions
        st.subheader("Resultados Simulación")
        col_res1, col_res2, col_res3 = st.columns(3)
        with col_res1: st.metric("Valor Final Proyectado", f"{final_balance:,.2f} €/$")
        with col_res2: st.metric("Aportaciones Totales", f"{total_contributions:,.2f} €/$")
        with col_res3: st.metric("Crecimiento Estimado", f"{total_growth:,.2f} €/$", delta=f"{total_growth/total_contributions*100 if total_contributions>0 else 0:.1f}%")

        # Gráfico
        st.subheader("Gráfico Crecimiento Proyectado")
        investment_history.index = pd.to_datetime(investment_history.index)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=investment_history.index, y=investment_history['Balance'], mode='lines', name='Valor Inversión', line=dict(color='green', width=2), hovertemplate='<b>Fecha</b>: %{x|%Y-%m-%d}<br><b>Balance</b>: %{y:,.2f} €/$<extra></extra>'))
        fig.update_layout(title=f'Crecimiento Inversión {selected_etf} (Predicción)', xaxis_title='Fecha', yaxis_title='Valor (€/$)', xaxis_tickformat='%Y-%m-%d', hovermode='x unified', legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
        st.plotly_chart(fig, use_container_width=True)
    except Exception as e_calc: st.error(f"Error cálculo/gráficación: {e_calc}"); st.text(traceback.format_exc())

# Footer
st.sidebar.markdown("---"); st.sidebar.info("Calculadora ilustrativa (predicciones Prophet). No garantía resultados.")

"""

# --- Nombre del archivo para guardar el código Streamlit ---
streamlit_app_filename = "calculadora_etf_app.py"

# --- Guardar el código Streamlit en un archivo .py ---
try:
    with open(streamlit_app_filename, "w", encoding="utf-8") as f:
        f.write(streamlit_app_code)
    print(f"\nCódigo de la aplicación Streamlit guardado en '{streamlit_app_filename}'")
except Exception as e_write:
    print(f"\nERROR al guardar el script de Streamlit: {e_write}")

In [ ]:
# --- Configuración e Inicio de ngrok ---

# 1. Reemplaza con tu Authtoken de ngrok.com
NGROK_AUTH_TOKEN = "2w5gkPnuxS2MaHakM0sGPAMNMdu_2dnPu3XzCXhSh1nQWin4j" # !!! CAMBIA ESTO !!!

# 2. Configurar ngrok
print("\nConfigurando ngrok...")
!ngrok authtoken {NGROK_AUTH_TOKEN}

# 3. Importar pyngrok
from pyngrok import ngrok
import time

# 4. Matar túneles previos (por si acaso)
print("Cerrando túneles ngrok existentes...")
ngrok.kill()
time.sleep(2) # Pequeña pausa

# 5. Iniciar Streamlit en segundo plano (asegúrate que el .py y .pkl existen)
print(f"\nIniciando Streamlit desde '{streamlit_app_filename}' en segundo plano...")
# Quita '&>/dev/null&' si necesitas ver errores de Streamlit directamente
!streamlit run {streamlit_app_filename} &>/dev/null&
print("Esperando unos segundos a que Streamlit arranque...")
time.sleep(10) # Aumentar si Streamlit tarda más en arrancar

# 6. Crear túnel ngrok
port = 8501 # Puerto estándar de Streamlit
print(f"\nCreando túnel ngrok hacia el puerto {port}...")
try:
    public_url = ngrok.connect(port)
    print("*"*50)
    print(f"Aplicación Streamlit debería estar lista.")
    print(f"Accede a ella a través de esta URL pública:")
    print(public_url)
    print("*"*50)
    print("(Puedes detener la ejecución de esta celda para cerrar el túnel)")
except Exception as e_ngrok:
    print(f"ERROR al iniciar ngrok: {e_ngrok}")
    print("Verifica tu Authtoken y que ngrok no esté ya ejecutando otro proceso conflictivo.")
    print("Intenta reiniciar el entorno de ejecución y volver a empezar.")

# Esta celda se mantendrá ejecutándose para mantener el túnel activo.
# Deténla manualmente (Ctrl+C o botón de stop) cuando termines.